<h1 style="text-align:center"><font face="Times New Roman">Final Project </font></h1>

<h2 style="text-align:center">
    <font face="Times New Roman">
        <b>Option 1: Geometric Moments and Neural Shape Analysis</b> 
    </font>
</h2>

<h3 style="text-align:center"><font face="Times New Roman">Chenxi Liu</font></h3> <h3 style="text-align:center"><font face="Times New Roman">929201838</font></h3>

<h1><strong><font face="Times New Roman">(I) Project Requirements</font></strong></h1>

 <p>
  <font face="Times New Roman" size=4>
      1. In this project, we are going to implement a deep neural network applied to point clouds and processes on the points’ coordinates for classification task, which is known as the classic <b>PointNet</b>.
  </font>
</p>

<p>
    <font face="Times New Roman" size=4>
      2. In order to better capture local structures, we also implement deep neural network architecture <b>Momenet</b>, which takes the geometry context in the form of geometric moments of 3D shapes into consideration and adds polynomial  functions to the origin point cloud coordinates.
    </font>
</p>

<p>
    <font face="Times New Roman" size=4>
    3. The main models we built in this project are as follows:<br>
        (1) Classic PointNet: <b>pointnet</b><br>
        (2) Classic Momenet (1st and 2nd order moments): <b>momenet</b><br>
        (3) 1st, 2nd, and 3rd order moments: <b>momenet3</b><br>
        (4) PointNet with Vertex Normals: <b>pointnet_vn</b><br>
        (5) Momenet with Vertex Normals: <b>momenet_vn2</b><br>
        (6) 1st, 2nd, and 3rd Momenet with Vertex Normals: <b>momenet_vn3</b><br>
        (7) PointNet with Harmonic Pre-lifting: <b>pointnet_hp</b>
    </font>
</p>

<h1><strong><font face="Times New Roman">(II) Implementation</font></strong></h1>

<h2><strong><font face="Times New Roman">1. Preprocessing</font></strong></h2>

<h3><strong><font face="Times New Roman">(1) Dataset</font></strong></h3>

- <p>
  <font face="Times New Roman" size=4>
       In the original paper, the authors evaluated PointNet on the ModelNet40 shape classification benchmark. It contains 12,311 models from 40 object categories, split into 9,843 training and 2,468 for testing.
       
  </font>
</p>

- <p>
  <font face="Times New Roman" size=4>
      Due to the large size of dataset and the long running time. We’ll use a smaller version of the same dataset instead: <b>ModelNet10</b>. It consists of objects from <b>10 categories</b>, 3,991 models for training and 908 for testing.
   </font>
</p>

- <p>
  <font face="Times New Roman" size=4>
      We define the class <b>PointCloudData(Dataset)</b> to preprocess the data from the train folders and test folders respectively to get the train and test dataset.<br>
      The default preprocessing include: <br>
        - Point Sampling,<br>
        - Point Normalization,<br>
        - Point Rotation,<br>
        - Add Noise to the points<br>
   </font>
</p>

In [ ]:
   def train_transforms():
        return torchvision.transforms.Compose([PointSampler(1024),
                                   Normalize(),
                                   RandRotation(),
                                   RandomNoise(),
                                   ToTensor()
                                 ])

- <p>
  <font face="Times New Roman" size=4>
       After preprocessing, we will get<br>
       - training dataset: train_ds (size: 3991)<br>
       - testing dataset: valid_ds (size: 908)<br> 
       where the sampled point cloud shape is $(N, C): torch.Size([1024, 3])$ <br>
      and the number of classes is: $10$.
  </font>
</p>

<h3><strong><font face="Times New Roman">(2) Read Files</font></strong></h3>

- <p>
   <font face="Times New Roman" size=4>
       The datasets consists of <b>.off</b> files which contains meshes represented by <em>triangular faces</em> and <em>vertices</em>. Each file contains $n$ vertices $V \in \mathbb{R}^3$ , where $n$ is the number of vertices, and $V=\{v_1, ..., v_n\}$, each point $v_i$ is given by its coordinates $(x_i, y_i, z_i)^T$.
   <\font>
<\p>
- <p>
   <font face="Times New Roman" size=4>
       So We defined <b>read_off(file)</b> to read the .off files.
   <\font>
<\p>

<h3><strong><font face="Times New Roman">(3) Point Sampling</font></strong></h3>

- <p>
   <font face="Times New Roman" size=4>
       If we read a mesh <b>bed.off</b>  for example, Most of the points of the origin point cloud are from the bent and rounded part of the mesh. So we almost can’t tell the origin shape given the origin point cloud. Since the points from the origin point cloud are not uniformly distributed, it would be difficult for the neural network to correctly classify them. <br>
   <\font>
<\p>
- <p>
   <font face="Times New Roman" size=4>
       In order to make the point cloud features more distinct and representative, we need to uniformly sample the points on the object’s surface.
   <\font>
<\p>
    
    
  <center>
      <img src="Figure/bed1.png" width="250" height="250" />
      <img src="Figure/bed2.png" width="250" height="250" />
      <br>
      <font face="Times New Roman" size=4>
      <div style="color:orange; 
                  display: inline-block;
                  color: black;
                  padding: 2px;">
                   &emsp;&emsp;&emsp; 
                      Origin Point Cloud  
                  &emsp;&emsp;&emsp;     
                      Uniformly Sampled Point Cloud
      </div>
           </font>
  </center>
    
- <p>
   <font face="Times New Roman" size=4>
       The way we did the point sampling is as follows:<br>
       (1) Calculate $areas$ of all faces in the mesh.<br>
       (2) Set $k$ fixed number of points: Since we will have dense layers in out neural network architecture, we will set a fixed number of points in a point cloud. The number in our case is $1024$.<br>
       (3) Assign probability: the probability of choosing a particular face is proportionally to its area.<br>
       (4) Sample faces:
   <\font>
<\p>


In [ ]:
    sampled_faces = (random.choices(faces, weights=areas, k=1024))

<p>
   <font face="Times New Roman" size=4>
          &ensp;&ensp;&ensp;(5) Sample points: Finally we will sample points on chosen faces for the point cloud of size $k$.
   <\font>
<\p>


<h3><strong><font face="Times New Roman">(4) Data Augmentation</font></strong></h3>

- <p>
   <font face="Times New Roman" size=4>
       Since the objects can have different sizes and can be placed in different parts of our coordinate system. We need further augment our sampled point cloud data.<br><br>
       (1) We’ll <b>translate</b> the object to the origin by subtracting mean from all its points and <b>normalizing</b> its points into a unit sphere, where $x, y, z \in [-1, 1]$. This will also meet the requirements for <em>Question(e)</em>: which requires us to rescale the point cloud input to fit inside the unit sphere in $\mathbb{R}^3$ in order to promise a well-behaved dynamic range in the harmonic case.<br>
       (2) In order to better augment the input data during training, we also randomly <b>rotate</b> objects around Z-axis and add <b>Gaussian noise</b>.
   <\font>
<\p>
    <center>
      <img src="Figure/bed3.png" width="250" height="250" />
      <br>
      <div style="color:orange; 
                  display: inline-block;
                  color: black;
                  padding: 2px;">Augmented Point Cloud</div>
  </center>

<h2><strong><font face="Times New Roman">2. Classic PointNet</font></strong></h2>

<h3><strong><font face="Times New Roman">(1) PointNet Architecture</font></strong></h3>

- <p>
   <font face="Times New Roman" size=4>
       The PointNet architecture pipeline commences with a transformer network for coping with input and feature transformations. Next it will input the transformed point cloud data into MLPs to generate a per point feature vector. Then, applies max pooling to generate global features that serve as a signature of the point cloud. Finally, fully connected layers produce output scores for each class.
   <\font>
<\p>  
  <center>
      <img src="Figure/PointNet_architecture.png" width="800" height="400" />
      <br>
      <font face="Times New Roman" size=4>
      <div style="color:orange; 
                  display: inline-block;
                  color: black;
                  padding: 2px;"> 
                     PointNet Architecture 
      </div>
           </font>
  </center>

<h3><strong><font face="Times New Roman">(2) PointNet Implementation</font></strong></h3>

<p>
   <font face="Times New Roman" size=4>
       1. The input size of the point cloud data is  (B×N×3), where <b>B</b> is the batch size, which is set to 32 in our case, and <b>N</b> is the number of points which equals to 1024, and <b>3</b> is the input channel. We first apply <b>input transform</b> to the point cloud data through T-Net, and multiply the 3×3 matrix we get from T-Net with the origin input. <br><br>
       The T-Net architecture is as follows, where k is the input channel and also the output matrix size we want to get:
   <\font>
<\p>  
  <center>
      <img src="Figure/Tnet.png" width="600" height="300" />
      <br>
      <div style="color:orange; 
                  display: inline-block;
                  color: black;
                  padding: 2px;"> 
                     T-Net Architecture 
      </div>
  </center>

<p>
   <font face="Times New Roman" size=4>
       2. Then, we’ll apply <b>feature transform</b> to the output from a (64, 64) mlp layer through T-Net, and multiply the  64×64 matrix we get with the output.
   <\font>
<\p>  
<p>
   <font face="Times New Roman" size=4>
       3. After applying a (64, 128, 1024) mlp layer, we’ll aggregates the point features by max pooling.
   <\font>
<\p>  
<p>
   <font face="Times New Roman" size=4>
       4. Finally, we'll apply a (512, 256,  10) fully connected layer and get the output scores for the 10 classes.
   <\font>
<\p>  

<h3><strong><font face="Times New Roman">(3) PointNet Result</font></strong></h3>

<p>
   <font face="Times New Roman" size=4>
       1. Here we will use the confusion matrix to better visualize the results we get for each model.<br>
       The confusion matrix we build has 3 axes.<br>
       (1) The x-axis is the predicted labels.<br>
       (2) The y-axis is the true labels. <br>
       (3) Inside the confusion matrix, we also have a heat map for the predictions we made, the higher the values the darker the color.<br><br>
       What we want to see when we’re training our network is the dark set of squares going down the diagonal, this is because the diagonal is where the predicted label is equal to the true label.<br>
       We have 2 types of confusion matrix:<br>
       (1) The value in the <b>unnormalized</b> confusion matrix represents the number of predictions we made for each class.<br>
       (2) The value in the <b>normalized</b> confusion matrix represents the true positive rate for each class, which equals <em>(correct number of predictions) $\div$ (total number of predictions)</em>
   <\font>
<\p>  
  <center>
      <img src="Figure/cm.png" width="400" height="400" />
      <img src="Figure/norm_cm.png" width="420" height="420" />
      <br>
      <div style="color:orange; 
                  display: inline-block;
                  color: black;
                  padding: 2px;"> 
                     PointNet Unnormalized(left) and Normalized(right) Confusion Matrix
      </div>
  </center>

<p>
   <font face="Times New Roman" size=4>
       2. The results of the PointNet validation accuracy for each epoch and Correct classification rate:
   <\font>
<\p>  
  <center>
      <img src="Figure/Acc/pointnet_acc.png" width="350" height="200" />
      <img src="Figure/TP/pointnet_tp.png" width="500" height="200" />
      
  </center>
    <p>
   <font face="Times New Roman" size=4>
       The mean validation accuracy of the PointNet is <b>$78.4$</b><br>
       The overall validation accuracy of the PointNet is <b>$85$</b>
   <\font>
<\p>  

<h2> <font face="Times New Roman">
       <strong> 3. Classic Momenet</strong> (1st and 2nd Order Moments)
     </font>
</h2>

<h3><strong><font face="Times New Roman">(1) Momenet Architecture</font></strong></h3>

- <p>
   <font face="Times New Roman" size=4>
       The baseline architecture of the suggested Momenet network is based on the PointNet architecture. The main differences between these two architecture mainly lays on the following 2 parts:<br>
       (1) 2nd Order-Layer: The Momenet architecture also adds a second Order-Layer before the MLP layers, which helps to add polynomial functions as part of the input domain.<br>
       (2) Spatial Transform: The Spatial Transformation is similar to the input transformation ( where T-Net input channel$=3$) in PointNet architecture. In Momenet Spatial-Trans, it also adds a 2nd-Order-Layer between The input data and the output $3 \times 3$ matrix.
   <\font>
<\p>  
  <center>
      <img src="Figure/Momenet_architecture.png" width="600" height="200" />
      <br>
      <font face="Times New Roman" size=4>
      <div style="color:orange; 
                  display: inline-block;
                  color: black;
                  padding: 2px;"> 
                     Momenet Architecture 
      </div>
           </font>
  </center>

- <p>
   <font face="Times New Roman" size=4>
       In the 2nd Order-Layer, besides the origin $(B \times N \times 3)$ input data, we also need to add <b>polynomial functions</b> and the <b>k nearest neighbors</b> of the original data as additional input features to the network. And in the 2nd Order Momenets, the polynomial functions includes $(x^2, y^2, z^2, xy, xz, yz)$. And K nearest neighbors for each point. So the whole Input size becomes $(B \times N \times K \times 12)$
   <\font>
<\p>  

<h3><strong><font face="Times New Roman">(2) Momenet Implementation</font></strong></h3>

- <p>
   <font face="Times New Roman" size=4>
       <b>Difficulties:</b> <br>
       In this part, I met some difficulties on implementing the <b>Spatial-Trans</b>. I first used the previous input transformation $Tnet(k=3)$ as Spatial-Trans and feed the output direct to the 2nd Order-Layers. But after training the model, the valid accuracies are between $18\% $ ~ $47\%$, which are quite low compared with the results we get from PointNet.<br>
       <b>Solutions: </b><br>
       - Then I added the 2nd Order-Layer between the original input data and the output matrix as shown in the paper. <br>
       - After that, I feed the output from the 2nd Order-Layer $(B \times N \times K \times C)$ , where $C$ is the total channels of the origin input data, polynomial functions and k nearest neighbors ($C=12$ in this case) to the same piplines as $Tnet$ but with $nn.Conv2d()$ in this setting. <br>
       - After getting the $(B \times 3 \times 3)$ matrix, I apply a batch mutiplication with the origin $(B \times N \times 3)$ input.
       - Then we feed the transformed $(B \times N \times 3)$ data to the MLP layers and the following piplines as shown in the architecture. This time the results become much better compared with the previous failure results.
   <\font>
<\p>  

- <p>
   <font face="Times New Roman" size=4>
       <b>Functions:</b> <br>
       The kernel functions we use in this part is <b>get_second()</b> and <b>KNN()</b> in the <b>Second_Order class</b>
   <\font>
<\p>  

<h3><strong><font face="Times New Roman">(3) Momenet Result</font></strong></h3>

<p>
   <font face="Times New Roman" size=4>
       Results of the 2nd Order Momenet validation accuracy for each epoch and Correct classification rate:
   <\font>
<\p>  
  <center>
      <img src="Figure/Acc/momenet_acc.png" width="350" height="200" />
      <img src="Figure/TP/momenet_tp.png" width="500" height="200" />
      
  </center>
 <p>
   <font face="Times New Roman" size=4>
       The mean validation accuracy of the 2nd Order Momenet is <b>$78.8$</b><br>
       The overall validation accuracy of the 2nd Order Momenet is <b>$87$</b>
   <\font>
<\p>  

<h2> <font face="Times New Roman">
       <strong> 4. Third Order Momenet</strong> (1st, 2nd and 3rd Order Moments)
     </font>
</h2>

<h3><strong><font face="Times New Roman">(1) 3rd Order Momenet Implementation</font></strong></h3>

- <p>
   <font face="Times New Roman" size=4>
       <b>Input shape changes:</b><br>
       The architecture for the 3rd Order Momenet is the same as the Classic Momenet. The only difference is the size of the input data. In the 3nd Order Momenets, besides the original input data, we also need to add the 2nd and the 3rd polynomial functions which includes $(x^2, y^2, z^2, xy, xz, yz, x^3, y^3, z^3, x^2y, x^2z, y^2x, y^2z, z^2x, z^2y, xyz)$ and k nearest neighbors $(B \times N \times K \times 22)$ as input. So the whole Input size becomes $(B \times N \times K \times 3).$
   <\font>
<\p>  

- <p>
   <font face="Times New Roman" size=4>
       <b>Functions:</b> <br>
       The kernel functions we use in this part is <b>get_third()</b> and <b>KNN()</b> in the <b>Second_Order class</b>
   <\font>
<\p>  

<h3><strong><font face="Times New Roman">(2) 3rd Order Momenet Result</font></strong></h3>

<p>
   <font face="Times New Roman" size=4>
       Results of the 3nd Order Momenet validation accuracy for each epoch and Correct classification rate:
   <\font>
<\p>  
  <center>
      <img src="Figure/Acc/momenet3_acc.png" width="350" height="200" />
      <img src="Figure/TP/momenet3_tp.png" width="500" height="200" />
      
  </center>
 <p>
   <font face="Times New Roman" size=4>
       The mean validation accuracy of the 2nd Order Momenet is <b>$79.8$</b><br>
       The overall validation accuracy of the 2nd Order Momenet is <b>$86$</b>
   <\font>
<\p>  

<h2> <font face="Times New Roman">
       <strong> 5. Adding Estimated Vertex Normals</strong> 
     </font>
</h2>

<h3><strong><font face="Times New Roman">(1) Vertex Normal Estimation</font></strong></h3>

- <p>
   <font face="Times New Roman" size=4>
       Since compute the vertex normals requires direct access to the mesh topology and it's not fairly applicable for our point cloud input. So we'll estimated each vertex normals by locally fitting a plane to each vertex. The reference of the theoretical derivation is down below:<br>
   <\font>
<\p>  

https://cs.nyu.edu/~panozzo/gp/04%20-%20Normal%20Estimation,%20Curves.pdf"/

- <p>
   <font face="Times New Roman" size=4>
       The main steps to estimate the consistant oriented vertex normal for a point are as follows: <br>
       (1) Find k nearest neighbors of the vertex point $x$.<br>
       (2) Compute the centroid of its' neighbor points:<br>
       <center>
       $m = \dfrac{1}{n}\sum_{i=1}^k x_i$
       </center>
       (3) Compute vectors from the centroid:<br>
       <center>
           $y_i = x_i - m$<br> 
           $Y = (y_1, y_2, ......y_k)$<br>
       </center>
       (4) Compute scatter matrix:<br>
       <center>$S = Y^TY$</center>
       (5) The plane normal <b>n</b> is the eigenvector of <b>S</b> with the smallest eigenvalue.
   <\font>
<\p>  

<h3><strong><font face="Times New Roman">(2) Add Estimated Vertex Normal to PointNet</font></strong></h3>

<p>
   <font face="Times New Roman" size=4>
       Results of the PointNet with vertex normals validation accuracy for each epoch and Correct classification rate:
   <\font>
<\p>  
  <center>
      <img src="Figure/Acc/pointnet_vn_acc.png" width="350" height="200" />
      <img src="Figure/TP/pointnet_vn_tp.png" width="500" height="200" />
      
  </center>
 <p>
   <font face="Times New Roman" size=4>
       The mean validation accuracy of the PointNet with vertex normals is <b>$74.5$</b><br>
       The overall validation accuracy of the PointNet with vertex normals is <b>$83$</b>
   <\font>
<\p>  

<h3><strong><font face="Times New Roman">(3) Add Estimated Vertex Normal to Classic Momenet</font></strong></h3>

<p>
   <font face="Times New Roman" size=4>
       Results of the 2nd Order Momenet with vertex normals validation accuracy for each epoch and Correct classification rate:
   <\font>
<\p>  
  <center>
      <img src="Figure/Acc/momenet_vn2_acc.png" width="350" height="200" />
      <img src="Figure/TP/momenet_vn2_tp.png" width="500" height="200" />
      
  </center>
 <p>
   <font face="Times New Roman" size=4>
       The mean validation accuracy of the 2nd Order Momenet with vertex normals is <b>$58.9$</b><br>
       The overall validation accuracy of the 2nd Order Momenet with vertex normals is <b>$76$</b>
   <\font>
<\p>  

<h3><strong><font face="Times New Roman">(4) Add Estimated Vertex Normal to 3rd Order Momenet</font></strong></h3>

<p>
   <font face="Times New Roman" size=4>
       Results of the 3rd Order Momenet with vertex normals validation accuracy for each epoch and Correct classification rate:
   <\font>
<\p>  
  <center>
      <img src="Figure/Acc/momenet_vn3_acc.png" width="350" height="200" />
      <img src="Figure/TP/momenet_vn3_tp.png" width="500" height="200" />
      
  </center>
 <p>
   <font face="Times New Roman" size=4>
       The mean validation accuracy of the 3rd Order Momenet with vertex normals is <b>$66.4$</b><br>
       The overall validation accuracy of the 3rd Order Momenet with vertex normals is <b>$79$</b>
   <\font>
<\p>  

<h2> <font face="Times New Roman">
       <strong> 6. Harmonic Geometric Pre-lifting</strong> 
     </font>
</h2>

<h3><strong><font face="Times New Roman">(1) Harmonic Pre-lifting</font></strong></h3>

- <p>
   <font face="Times New Roman" size=4>
       The harmonic pre-lifting we chose in this case are:<br>
       $sin(x),sin(y),sin(z), cos(x), cos(y), cos(z), sin(2x),sin(2y),sin(2z), cos(2x), cos(2y), cos(2z)$
   <\font>
<\p>  

<p>
   <font face="Times New Roman" size=4>
       Results of the PointNet with harmonic pre-lifting validation accuracy for each epoch and Correct classification rate:
   <\font>
<\p>  
  <center>
      <img src="Figure/Acc/pointnet_hp.png" width="350" height="200" />
      <img src="Figure/TP/pointnet_hp.png" width="500" height="200" />
      
  </center>
 <p>
   <font face="Times New Roman" size=4>
       The mean validation accuracy of the PointNet with harmonic pre-lifting is <b>$77.6$</b><br>
       The overall validation accuracy of the PointNet with harmonic pre-lifting is <b>$86$</b>
   <\font>
<\p>  

<h1><strong><font face="Times New Roman">(III) Analysis and Conclusions </font></strong></h1>

<p>
   <font face="Times New Roman" size=4>
       <b>Analysis:</b> <br>
       1. The line charts and the average of the valid accuracies for 15 epochs of training for PointNet, 2nd Order Momenet, 3rd Order Momenet, and PointNet with harmonic pre-lifting are as follows:
   <\font>
<\p>  
  <center>
      <img src="Figure/Acc/pmmp_acc.png" width="600" height="600" />    
      <img src="Figure/Acc/pmmp_avg.png" width="450" height="600" />
  </center> 

<p>
   <font face="Times New Roman" size=4>
       (1) The average valid accuracies for Classic PointNet and Momenet in this case are $78.4$ and $78.8$ respectively, which are lower than the results from the origin paper, where average accuracies for PointNet is $86.2$ and Momenet is $90.3$. The possible reasons may first due to the smaller dataset we used for training, we use a smaller dataset ModelNet10 compared with the ModelNet40 from the origin paper. And it may also due to the small number of epochs we used to train the network, we used $15$ epochs in our case, however in the origin paper, the author used $250$ epochs.<br><br>
       (2) Though we didn't get to the high valid accuracies as shown in the paper, we still find some similarities between our result and the original result.<br>
       - For example, the mean valid accuracy of Momenet is higher than the mean valid accuracy of the classic PointNet, which shows the same trend as the paper does. Apart from that, after adding higher order of polynomial functions to the classic Momenet, the mean valid accuracy increased as expected, which can be seen from the valid accuracy value we get in the case of Momenet3 $79.8$ and $78.8$.
   <\font>
<\p>  

<p>
   <font face="Times New Roman" size=4>
       2. The line charts of 10 epochs of training for PointNet_vn, Momenet_vn2 and Momenet_vn3 are as follows:
   <\font>
<\p>  
  <center>
      <img src="Figure/Acc/pmm_vn.png" width="600" height="600" />    
  </center> 

- <p>
   <font face="Times New Roman" size=4>
       3. The line charts of 10 epochs of training for all models are as follows:
   <\font>
<\p>  
  <center>
      <img src="Figure/Acc/six.png" width="600" height="600" />    
  </center> 
- <p>
   <font face="Times New Roman" size=4>
       We can see from the figure that the overall performance of Momenet outperforms other models. <br>And the results from models which also add the estimated vertex normals as input are less accurate than models didn't add estimated vertex normals, which we deduce that the vertex normals we estimated may not accurate enough, which may mislead the classification task for the network during training.
   <\font>
<\p>  

<p>
   <font face="Times New Roman" size=4>
       4. The tables for the true positive rate for all classes in each model are shown below:
   <\font>
<\p>  
  <center>
      <img src="Figure/Acc/table.png" width="1000" height="800" />    
  </center> 

- <p>
   <font face="Times New Roman" size=4>
       From the table we can see that, the Momenet model has the best mean true positive predicting rate for all the classes compared with other models.<br>
       However, our model sometimes confuses dressers with nightstands, desks with tables which is also quite understandble.
   <\font>
<\p>  

<p>
   <font face="Times New Roman" size=4>
       <b>Conclusions:</b><br>
       Though the overall performance and the accuracy results we get from our model are worse compared with the results from the origin paper. The overall results we get from our model still indicated that lifting the input dataset from $\mathbb{R}^3$ to $\mathbb{R}^9$ or even higher dimensions will increase the classification accuracy of the network after training.<br> Besides, the performance of the classic Momenet model we implemented outperforms other models.
   <\font>
<\p>  